Convert real value in dataset

In [ ]:
import pandas as pd

def convert_type(df: pd.DataFrame, types: pd.DataFrame):
  for idx, column in enumerate(df.columns):
    if (types[idx] == 'r'):
      df[column] = pd.to_numeric(df[column])


Import iris file

In [ ]:
iris_file = pd.read_csv('iris.tmls')

iris = pd.DataFrame(iris_file)

# Extract row which inform on the type of features
iris_types = iris.iloc[0]

# Remove of the table the row which inform on the type of features
iris = iris.drop(iris.index[0])

# Convert real value columns into numeric values
convert_type(iris, iris_types)

iris.info()

Import Wine file

In [ ]:
wine_file = pd.read_csv('wine.tmls')

wine = pd.DataFrame(wine_file)

wine_types = wine.iloc[0]

wine = wine.drop(wine.index[0])

convert_type(wine, wine_types)

wine.info()

Import adult file

In [ ]:
adult_file = pd.read_csv('adult.tmls')

adult = pd.DataFrame(adult_file)

adult_types = adult.iloc[0]

adult = adult.drop(adult.index[0])

convert_type(adult, adult_types)

adult.info()

Compute Gini index to estimate disorder : Gini index is used instead of entropy because it gives similar results almost everytime for classifier and is less computationally expensive

In [ ]:
def gini_index(data, classes: list):
  data_len = len(data)
  if (data_len == 0):
    return 0
  sum_classes = 0
  for class_name in classes:
    sum_classes += (data.count(class_name) / data_len)**2
  return 1 - sum_classes

# Estimate the disorder of the left and the right nodes depending on their relative size
def split_gini_index(classes_splited, classes, dataset_size):
  l_relative_size, r_relative_size = len(classes_splited["left"]) / dataset_size, len(classes_splited["right"]) / dataset_size
  l_gini, r_gini = gini_index(classes_splited["left"], classes) * l_relative_size, gini_index(classes_splited["right"], classes) * r_relative_size
  return l_gini + r_gini

Split a dataset in two nodes by a certain feature and a certain value in order to have the less disorder

In [ ]:
# Return the separation values for each column/feature :
# - mean of the feature for real-value column
# - all the different nominal value for nominal column ()
def get_col_values(dataset, col_size, dataset_size):
  col_values = []
  for col in range(col_size - 1):
    col_values.append([])
  for row in range(dataset_size):
    for col in range(col_size - 1):
      if ((not type(dataset[row][col]) is str) or (not dataset[row][col] in col_values[col])):
        col_values[col].append(dataset[row][col])
  for col in range(col_size - 1):
    if not (type(col_values[col][0]) is str):
      col_values[col] = [sum(col_values[col]) / len(col_values[col])]
  return col_values

# Split rows into 2 node depending the value of the row
def split_dataset_in_two(dataset, val, col):
  splited = {"left": [], "right": []}
  if (type(val) is str):
    for row in dataset:
      if (row[col] != val):
        splited["left"].append(row)
      else:
        splited["right"].append(row)
  else:
    for row in dataset:
      if (row[col] < val):
        splited["left"].append(row)
      else:
        splited["right"].append(row)
  return splited

# Find the split with the less disorder
def find_best_split(dataset, col_size, classes):
  b_gini = 1
  dataset_size = len(dataset)
  col_values = get_col_values(dataset=dataset, col_size=col_size, dataset_size=dataset_size)
  for col in range(col_size - 1):
    for col_val in col_values[col]:
      splited = split_dataset_in_two(dataset, col_val, col)
      classes_splited = {"left": [row[-1] for row in splited["left"]],
                          "right": [row[-1] for row in splited["right"]]}
      gini = split_gini_index(classes_splited, classes, dataset_size)
      if (gini < b_gini):
        b_gini = gini
  return col, col_val, splited


Decision tree classifier

In [ ]:
class cell:
  col = 0
  value = ''
  dataset = []
  left = {}
  right = {}
  end = False

# Return classes of the dataset
def get_classes(dataset):
  classes = []
  for i in range(len(dataset)):
    if not (dataset[i][-1] in classes):
      classes.append(dataset[i][-1])
  classes.sort()
  return classes

# Fill node with his left and right child, the feature and the value used to split
def update_cell(node, col, value, splited):
  node.col = col
  node.value = value
  node.left = cell()
  node.right = cell()
  node.left.dataset, node.right.dataset = splited["left"], splited["right"]

# Creating the leaf of the tree and save the most represented class as the value class of the leaf
def end_node(node: cell, lst):
  node.end = True
  node.value = max(set(lst), key=lst.count)

# Build decision tree node by node by splitting the node in two thanks to gini index in each iteration
def build_decision_tree(node: cell, col_size, classes, depth, max_depth, min_records):
  # End tree when max_depth is met
  if (depth == max_depth):
    end_node(node, [row[-1] for row in node.dataset])
    return
  col, value, splited = find_best_split(node.dataset, col_size, classes)
  update_cell(node, col, value, splited)
  if (len(node.left.dataset) > min_records):
    build_decision_tree(node.left, col_size, classes, depth + 1, max_depth, min_records)
  # End tree when not enough data in dataset
  else:
    end_node(node.left, [row[-1] for row in node.dataset])
  if (len(node.right.dataset) > min_records):
    build_decision_tree(node.right, col_size, classes, depth + 1, max_depth, min_records)
  # End tree when not enough data in dataset
  else:
    end_node(node.right, [row[-1] for row in node.dataset])
  # Free dataset to free memory with useless information
  node.dataset = []

# Predict the class of a row by searching in the tree depending its features values
def predict_class(node: cell, row):
  if (node.end == True):
    return node.value
  if (type(node.value) is str):
    if (row[node.col] != node.value):
      return predict_class(node.left, row)
    else:
      return predict_class(node.right, row)
  else:
    if (row[node.col] < node.value):
      return predict_class(node.left, row)
    else:
      return predict_class(node.right, row)

# Create decison tree with train and predict class from test row thanks to this tree
def decision_tree_learner(train, test, max_depth, min_records):
  predictions = []
  col_size = len(train[0])
  classes = get_classes(train + test)
  root = cell()
  root.dataset = train
  col, value, splited = find_best_split(train, col_size, classes)
  update_cell(root, col, value, splited)
  build_decision_tree(root, col_size, classes, 1, max_depth, min_records)
  for row in test:
    predictions.append(predict_class(root, row))
  return predictions


Cross validation

In [ ]:
from random import randrange

# Split randomly dataset in K folds
def split_dataset(dataset: pd.DataFrame, k_folds: int):
  fold_size = int(len(dataset.index) / k_folds)
  folds = []
  for _ in range(k_folds):
    fold = []
    for _ in range(fold_size):
      idx = randrange(len(dataset.index))
      fold.append(dataset.iloc[idx].to_list())
      dataset = dataset.drop(dataset.index[idx])
    folds.append(fold)
  return folds

def get_accuracy(results: list, predictions: list):
  accuracy = 0
  for i in range(len(predictions)):
    if (predictions[i] == results[i]):
      accuracy += 1
  accuracy = accuracy / len(predictions)
  return accuracy

def cross_validation(dataset: pd.DataFrame, k_folds: int, max_depth=5, min_records=10):
  dataset_copy = dataset.copy()
  folds = split_dataset(dataset_copy, k_folds)
  accuracy_array = []
  for idx in range(k_folds):
    results = [row[-1] for row in folds[idx]]
    train =  [row for fold in folds[:idx] for row in fold] + [row for fold in folds[idx + 1 :] for row in fold]
    predictions = decision_tree_learner(train=train, test=folds[idx], max_depth=max_depth, min_records=min_records)
    accuracy = get_accuracy(results, predictions)
    accuracy_array.append(accuracy)
  return accuracy_array

Iris decision learner accuracy (accuracy is significantly better when we have only two target class)

In [ ]:
cross_validation(iris, 10)

Wine decision learner accuracy (accuracy is significantly better when we have only two target class)

In [ ]:
cross_validation(wine, 10)

Adult decision learner accuracy (take approximately 2min)

In [ ]:
cross_validation(adult, 10)